In [1]:
clientId = "9e443dbb-73f2-4e20-bf4d-fffcd86cd926"
clientKey = "1]1mmsSjBHe0PXzD8trcJsxWh.iOUa_/"
tenantId = "fd89f191-166e-474f-a80f-af83c8f5e17e"
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientId,
           "fs.azure.account.oauth2.client.secret": clientKey,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/"+tenantId+"/oauth2/token"}

# # Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
 source = "abfss://rawdata@team9adls.dfs.core.windows.net/",
 mount_point = "/mnt/rawdata",
 extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-730143685605714> in <module> () 
 12 source = "abfss://rawdata@team9adls.dfs.core.windows.net/" , 
 13 mount_point = "/mnt/rawdata" , 
 ---> 14 extra_configs = configs)
 
 /local_disk0/tmp/1574175157772-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 306 exc . __context__ = None 
 307 exc . __cause__ = None 
 --> 308 raise exc 
 309 return f_with_exception_handling 
 310 

 ExecutionError : An error occurred while calling o225.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/rawdata; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/rawdata
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.GeneratedMethodAccessor410.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/rawdata
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:205)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:307)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:201)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:298)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:276)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:50)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:67)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:67)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:46)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:417)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.sc

In [2]:
import pandas as pd

Sales_Addresses_FP = "/mnt/rawdata/cloudsales/dboAddresses.csv"
Streaming_Addresses_FP = "/mnt/rawdata/cloudstreaming/dboAddresses.csv"

Sales_Addresses_Raw = sqlContext.read.parquet(Sales_Addresses_FP)
Streaming_Addresses_Raw = sqlContext.read.parquet(Streaming_Addresses_FP)

Sales_Addresses_Raw = Sales_Addresses_Raw.toPandas()
Streaming_Addresses_Raw = Streaming_Addresses_Raw.toPandas()

Sales_Addresses_DF = [Sales_Addresses_Raw, Streaming_Addresses_Raw]
Cloud_Addresses = pd.concat(Sales_Addresses_DF)

Cloud_Addresses['CreatedDate'] = pd.to_datetime(Cloud_Addresses['CreatedDate'], errors='coerce')
Cloud_Addresses['UpdatedDate'] = pd.to_datetime(Cloud_Addresses['UpdatedDate'], errors='coerce')

Cloud_Addresses.AddressLine2 = Cloud_Addresses.AddressLine2.astype(str)
Cloud_Addresses.ZipCode = Cloud_Addresses.ZipCode.astype(str)

display(Cloud_Addresses)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate
d06fde5d-43bd-4b5c-b565-002b38303f32,1dc447c9-06e9-4741-82ba-917dd1dd8b52,982 Fifth Ave,None,Portland,OR,85101,2017-01-01T00:00:00.000+0000,2017-09-12T00:00:00.000+0000
778daea8-e9e3-4d36-bea5-0071afa39efa,e03ed1c7-4d61-4827-9a3a-451ad032c6a8,817 Cherry St,None,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-04-07T00:00:00.000+0000
0ec3dc5b-d562-469f-959e-009560f0a197,16843ab0-5d12-49d4-b2a6-d9602d5bf750,924 Pine St,None,San Diego,CA,98101,2017-07-16T00:00:00.000+0000,2017-11-15T00:00:00.000+0000
5d13fb39-67cf-4cd6-8502-00a56d3037c7,b2f5eacf-7f6c-4009-9198-34ade9f7338e,926 Eighth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-01-22T00:00:00.000+0000
7e1dab8e-15f0-43d1-b099-00c4c5275272,75f6b4da-e62e-4735-97cb-00b5278b8275,445 Tenth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-06-03T00:00:00.000+0000
df5a8273-2500-48aa-a3e7-010e0054aa68,9a64f899-41a4-46b1-8320-f3193cae7e33,828 First Ave,None,Washington,DC,21001,2017-10-22T00:00:00.000+0000,2017-12-26T00:00:00.000+0000
e6b830be-4286-4d0a-8d90-011054ee7559,2a3b101e-4c54-47a7-b62a-3a989a869e67,921 Ninth Ave,None,Seattle,WA,92101,2017-01-01T00:00:00.000+0000,2017-08-13T00:00:00.000+0000
f99541ee-0f3b-44a8-955e-014cacd525a9,5f5ef893-9f73-4d67-889c-1d5adc42bad6,777 Ninth Ave,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-07-23T00:00:00.000+0000
eb70b894-a01d-45ff-8fbd-01962bbfe8fc,20a75138-e786-4c3e-a8b7-61d5f938fee7,93 Pine St,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-05-03T00:00:00.000+0000
4d33d894-e638-4f46-8853-01eab937c79f,5a2defe9-bb17-4c84-861b-6f69f5e479e6,884 Tenth Ave,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-04-03T00:00:00.000+0000


In [3]:
VanCustomers_FP = "/mnt/rawdata/vanarsdell/dboCustomers.txt"

VanCustomers_raw = spark.read.json(VanCustomers_FP)
VanCustomers_df = VanCustomers_raw.toPandas()

VanAddresses = VanCustomers_df[['CustomerID', 'AddressLine1', 'AddressLine2', 'City', 'State', 'ZipCode', 'CreatedDate', 'UpdatedDate']]

VanAddresses['CreatedDate'] = pd.to_datetime(VanAddresses['CreatedDate'], errors='coerce')
VanAddresses['UpdatedDate'] = pd.to_datetime(VanAddresses['UpdatedDate'], errors='coerce')

VanAddresses.AddressLine2 = VanAddresses.AddressLine2.astype(str)
VanAddresses.ZipCode = VanAddresses.ZipCode.astype(str)

VanAddresses.insert(0, 'AddressID', 'None')

display(VanAddresses)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate
None,9940f4f7-09f4-4b6f-8bea-00adc5d08e97,232 Cherry St,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-06-30T00:00:00.000+0000
None,d91caa4a-07dc-45ae-aa75-011b3ea8466b,780 Second Ave,None,Austin,TX,75101,2017-04-22T00:00:00.000+0000,2017-10-09T00:00:00.000+0000
None,2d268832-845e-49af-9287-013e0cad582a,401 Tenth Ave,None,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-08-11T00:00:00.000+0000
None,2ff332f3-25fc-46ed-aff6-016e9d1535c5,335 Cherry St,None,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-08-30T00:00:00.000+0000
None,179b2d93-b012-437e-94eb-0178d7571226,430 Fourth Ave,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-09-29T00:00:00.000+0000
None,0b6620cb-5db7-446e-9932-02207765703d,692 Fourth Ave,None,Washington,DC,21001,2017-05-08T00:00:00.000+0000,2017-06-04T00:00:00.000+0000
None,d8c55ce2-9111-44bc-b7e9-022ea4e68923,733 Fourth Ave,None,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-07-31T00:00:00.000+0000
None,b50e3b29-c731-4e12-bb44-044c058e1322,449 Maple St,None,Seattle,WA,92101,2017-05-29T00:00:00.000+0000,2017-08-17T00:00:00.000+0000
None,b52f0b8d-9e38-4fd1-b984-047e026b4130,335 Eighth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-10-19T00:00:00.000+0000
None,c5103deb-cf9c-4c4e-8b4b-05acda213781,98 Sixth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-08-28T00:00:00.000+0000


In [4]:
CoffeeCustomers_FP = "/mnt/rawdata/fourthcoffee/Customers.csv"

CoffeeCustomer_raw = spark.read.csv(CoffeeCustomers_FP, header=True)

CoffeeCustomers_df = CoffeeCustomer_raw.toPandas()

CoffeeAddresses = CoffeeCustomers_df[['CustomerID', 'AddressLine1', 'AddressLine2', 'City', 'State', 'ZipCode', 'CreatedDate', 'UpdatedDate']]

CoffeeAddresses['CreatedDate'] = pd.to_datetime(CoffeeAddresses['CreatedDate'], errors='coerce')
CoffeeAddresses['UpdatedDate'] = pd.to_datetime(CoffeeAddresses['UpdatedDate'], errors='coerce')
CoffeeAddresses.AddressLine2 = CoffeeAddresses.AddressLine2.astype(str)
CoffeeAddresses.ZipCode = CoffeeAddresses.ZipCode.astype(str)

CoffeeAddresses.loc[CoffeeAddresses.AddressLine2 == 'nan', 'AddressLine2'] = 'None'

CoffeeAddresses.insert(0, 'AddressID', 'None')

display(CoffeeAddresses)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate
None,B09477DC-5DCD-412A-AC13-01606E390264,268 Tenth Ave,NULL,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-07-16T00:00:00.000+0000
None,3B3D46CB-FDB6-494F-82DF-02101A02EDFD,120 Cherry St,NULL,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-11-07T00:00:00.000+0000
None,F7596520-CD3E-490E-A8EF-0349B36EAAE1,981 Eighth Ave,NULL,San Diego,CA,98101,2017-05-23T00:00:00.000+0000,2017-10-12T00:00:00.000+0000
None,0A58BEA6-B3BD-4867-9F1D-035046FFB03C,654 Sixth Ave,NULL,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-09-14T00:00:00.000+0000
None,D05444AD-4558-40BA-A022-047D53E8ACF8,263 Maple St,NULL,Seattle,WA,92101,2017-01-01T00:00:00.000+0000,2017-10-07T00:00:00.000+0000
None,D199E279-E223-4817-9070-04E1F3285786,732 Maple St,NULL,Portland,OR,85101,2017-04-24T00:00:00.000+0000,2017-06-21T00:00:00.000+0000
None,9C58BA57-CE0A-4613-B08B-0544EF2BCCC8,50 Fifth Ave,NULL,Seattle,WA,92101,2017-02-02T00:00:00.000+0000,2017-10-30T00:00:00.000+0000
None,3BA863F6-C9DD-4AC9-A85D-05D6F5F72383,808 Fourth Ave,NULL,Seattle,WA,92101,2017-01-01T00:00:00.000+0000,2017-07-20T00:00:00.000+0000
None,550581B7-88E5-4466-A314-06A1CE29ACFD,145 Dogwood St,NULL,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-06-10T00:00:00.000+0000
None,618E11A2-E3F0-4A20-83B9-0712E132D4A4,751 Fifth Ave,NULL,Austin,TX,75101,2017-05-30T00:00:00.000+0000,2017-07-28T00:00:00.000+0000


In [5]:
Cloud_Addresses['SourceSystem'] = '1'
VanAddresses['SourceSystem'] = '2'
CoffeeAddresses['SourceSystem'] = '3'
addresses_frame = [Cloud_Addresses, VanAddresses, CoffeeAddresses]

all_addresses = pd.concat(addresses_frame)

display(all_addresses)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate,SourceSystem
d06fde5d-43bd-4b5c-b565-002b38303f32,1dc447c9-06e9-4741-82ba-917dd1dd8b52,982 Fifth Ave,None,Portland,OR,85101,2017-01-01T00:00:00.000+0000,2017-09-12T00:00:00.000+0000,1
778daea8-e9e3-4d36-bea5-0071afa39efa,e03ed1c7-4d61-4827-9a3a-451ad032c6a8,817 Cherry St,None,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-04-07T00:00:00.000+0000,1
0ec3dc5b-d562-469f-959e-009560f0a197,16843ab0-5d12-49d4-b2a6-d9602d5bf750,924 Pine St,None,San Diego,CA,98101,2017-07-16T00:00:00.000+0000,2017-11-15T00:00:00.000+0000,1
5d13fb39-67cf-4cd6-8502-00a56d3037c7,b2f5eacf-7f6c-4009-9198-34ade9f7338e,926 Eighth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-01-22T00:00:00.000+0000,1
7e1dab8e-15f0-43d1-b099-00c4c5275272,75f6b4da-e62e-4735-97cb-00b5278b8275,445 Tenth Ave,None,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-06-03T00:00:00.000+0000,1
df5a8273-2500-48aa-a3e7-010e0054aa68,9a64f899-41a4-46b1-8320-f3193cae7e33,828 First Ave,None,Washington,DC,21001,2017-10-22T00:00:00.000+0000,2017-12-26T00:00:00.000+0000,1
e6b830be-4286-4d0a-8d90-011054ee7559,2a3b101e-4c54-47a7-b62a-3a989a869e67,921 Ninth Ave,None,Seattle,WA,92101,2017-01-01T00:00:00.000+0000,2017-08-13T00:00:00.000+0000,1
f99541ee-0f3b-44a8-955e-014cacd525a9,5f5ef893-9f73-4d67-889c-1d5adc42bad6,777 Ninth Ave,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-07-23T00:00:00.000+0000,1
eb70b894-a01d-45ff-8fbd-01962bbfe8fc,20a75138-e786-4c3e-a8b7-61d5f938fee7,93 Pine St,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-05-03T00:00:00.000+0000,1
4d33d894-e638-4f46-8853-01eab937c79f,5a2defe9-bb17-4c84-861b-6f69f5e479e6,884 Tenth Ave,None,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-04-03T00:00:00.000+0000,1


In [6]:
all_addresses = spark.createDataFrame(all_addresses)
all_addresses.coalesce(1).write.mode("overwrite").format("com.databricks.spark.csv").option("header","true").csv("/mnt/rawdata/stagingdata/AllAddresses/")